#  11. Solidity 제어, 데이터구조

* Last Updated 20200606SAT1200 20190120SUN0000 20180707SAT1300 20170112
* todo: 1) ```watch``` 2) library ```myLib+StringUtils``` 3) 계정에서 발생한 거래의 목록? 4) registry

## 9.1 학습내용

### 9.1.1 목표
 
* solidity 배우기

### 9.1.2 목차

* 9.6 [제어](#9.6)
    * 문제 Hello World Contract - 이름정하고, 헬로우 할 때 알아내기
* 9.7 데이터구조
    * 9.7.1 Array
    * 9.7.2 [mapping](#S.5.2)
    * 실습 Members
    * 실습 BankV4
* 9.8 [Exception](#9.6)


## 9.6 제어
<a id='11.4'></a>


### 9.6.1 반복문
블록체인에서의 프로그램은 명령문마다 비용이 발생하는 특성을 가지고 있기 때문에, 반복을 하게되면 비용이 발생하게 된다.
gas가 반복마다 소비되고 한도 gasLimit를 초과하게 되면 실행이 중지될 수 밖에 없다.

* for문
```python
function getSum() public returns(uint) {
    uint sum=0;
    for(uint i=0;i<10;i++)
        sum+=i;
    return sum;
}
```

* while문
```python
function getSum() public returns(uint) {
    uint sum=0;
    uint i=0;
    do {
        sum+=i;
        i++;
    } while(i<10);
    return sum;
}
```

반복횟수에 따라 gas비용이 급증하는 것을 볼 수 있다.
특히 sum 변수를 메모리에 두는 경우에 비해, storage에 (state variable) 두는 경우 비용이 매우 높았다.
메모리를 사용하는 것에 비해 Storage가 많이 비용이 많이 든다.

반복 횟수 | sum을 로컬에 두는 경우 | sum을 state variable에 두는 경우
-----|-----|-----
10 | 약 30,000 (23845 + 6773)  |  약 64,000 (43045 + 21773)
100 | 약 78,000 (49345 + 28073)  |  약 368,000 (192881 + 175809)
1000 | 약 203,000 (112345 + 91073) | 약 3,408,000 (1712981 + 1695909)
10000 | REMIX가 죽고, 다시 실행 | 좌동

In [10]:
%%writefile src/ControlFlowTest.sol
pragma solidity ^0.6.0;
contract ControlFlowTest {
    uint sum;
    constructor() public {
        sum = 0;
    }
    function doForLoop() public {
        //sum = 0;
        uint sumLocal = 0;
        for(uint i = 0; i < 101; i++) {
            sumLocal += i;
        }
        sum = sumLocal;
    }
    function doForLoop2() public {
        sum = 0;
        //uint sumLocal = 0;
        for(uint i = 0; i < 101; i++) {
            //sumLocal += i;
            sum += i;
        }
        //sum = sumLocal;
    }
    function getSum() view public returns(uint) {
        return sum;
    }
}

Writing src/ControlFlowTest.sol


* switch, goto를 제외하고, 다른 언어에서 제공하는 if, else, while, do, for, break, continue, return, ?

```python
if(condition) {
    ...
} else if {
    ...
}
```

* 삼항 연산자 ternary operator
```python
betAmount=betAmount>amount ? amount : betAmount;
```

## 실습: 인사하는 컨트랙

"Hello"를 출력하는 프로그램을 수정해서 if문을 사용해 보자.
modifier를 사용하면 조건문을 사용하지 않을 수 있다.
조건문에서 비교문을 사용하게 되는데, ```string```은 참조타입이라 비교를 하면 주소를 비교하게 된다. 어떻게 해야 하는지 생각해보자.

```python
contract Hello1 {
    function sayHello() 
    modifier isOwner()
    function setHello()
    function compareTo(string memory _str)
    function getBalance()
}
```

###  단계 1: 컨트랙 개발

Line | 설명
-----|-----
1 | major version 6 이상 최신으로 컴파일
2 | contract 명. 파일명과 반드시 일치할 필요가 없다.
3 | 속성 ```hello```. 누구나 사용할 수 없도록 ```private```
4 | 속성 ```address```. ```private```으로 자신만 사용할 수 있게 선언.
5 | ```event```로서 주소, 문자열 인자를 출력한다. Solidity에는 print문이 별도로 없다. 문자열을 반환하거나 ```event```문으로 출력할 수 있다.
6 ~ 9 | 생성자로서 외부에서 인자를 받아 속성을 초기화. ```constructor```는 ```public``` 또는 ```internal```로 선언해야 한다. 속성 값을 넣어준다. ```msg```는 전역변수로서 ```msg.sender```는 전송자의 계정주소를 말한다. 생성자의 ```msg.sender```를 컨트랙의 소유주 ```owner```로 등록한다.
10 ~ 12 | ```sayHello()``` 함수는 속성 값을 읽는다. 따라서 ```view```로 선언한다. 인자는 함수 내에서만 잠시 사용하므로 ```memory```로 선언
13 ~ 18 | ```modifier``` 함수. ```msg.sender```와 ```owner```가 일치하지 않으면 예외처리한다. 아니면 계속 실행.
19 ~ 29 | ```setHello()``` 함수는 ```event```함수를 가지고 있으므로 ```view```로 선언하지 않는다. ```view```와 ```event```는 같이 사용할 수 없다. ```event```는 상태를 변경하기 때문이다.
30 ~ 37 | ```compareTo()```함수는 입력문자열과 hello를 비교한다. 문자열은 참조값이기 때문에 비교할 경우 참조를 비교하게 되므로 주의를 해야한다. hash로 비교하고 있다.
38 ~ 40 | 잔고를 구한다.


In [2]:
%%writefile src/Hello1.sol
pragma solidity ^0.6.0;
contract Hello1 {
    string private hello;
    address private owner;
    event PrintLog(address addr, string s);
    constructor(string memory _hello) public {
        hello = _hello;
        owner = msg.sender;
    }
    function sayHello() view public returns(string memory) {
        return hello;
    }
    modifier isOwner() {
        if (msg.sender != owner) {
            revert();
        }
        _; //continue executing rest of method body
    }
    function setHello() public {
        string memory s = "";
        if (msg.sender == owner) {
            s = "Hello";
            emit PrintLog(msg.sender, s);
        } else {
            s = "Olleh";
            emit PrintLog(msg.sender, s);
        }
        hello = s;
    }
    function compareTo(string memory _str) view public returns(bool) {
        bool isEqual = false;
        //if (hello == _str) {
        if (keccak256(bytes(hello)) == keccak256(bytes(_str))) {
            isEqual = true;
        }
        return isEqual;
    }
    function getBalance() view public isOwner returns(uint) {
        return address(this).balance;
    }
}


Overwriting src/Hello1.sol


### 단계 2: 컴파일

In [2]:
!solc src/Hello1.sol --combined-json abi,bin > src/Hello1.json

### 단계 3: 배포

bytecode는 16진수이므로 앞에 ```0x```를 붙여준다.
gas를 지급해야 하는 등 비용이 발생하는 경우에는 반드시 web3.personal.unlockAccount()를 해주어야 한다.
REMIX를 사용하면, Web3 deploy를 복사해서 사용할 수 있다. abi, bin을 넣은 javascript명령문을 발견할 수 있다.

In [7]:
%%writefile src/Hello1DeployFromFile.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/Hello1.json");
var _json = JSON.parse(_str)
//var _abiArray = JSON.parse(_json.contracts.sHello2.abi);
var _abiArray = JSON.parse(_json.contracts["src/Hello1.sol:Hello1"].abi);
//var _bin = _json.contracts.sHello2.bin;
var _bin="0x" + _json.contracts["src/Hello1.sol:Hello1"].bin;

//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin, arguments: ["Hello from web3"]})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()


Writing src/Hello1DeployFromFile.js


In [8]:
!node src/Hello1Deploy.js

Deploying the contract from 0x28E9D5b10083b74406b50cbb7B48a299BA6C8Be1
hash: 0xf1d660888318cc1428289aa1481a2fd1bf8e968eed9f408c9d7192656dccfa35
---> The contract deployed to: 0xA43f621f0bAFfce634E43858Fb72809f050049E4


개인망을 띄우고, node로 Hello1Deploy.js를 실행하면 화면좌측에 보이듯이 transaction이 생성되고 그 hash값 이 주어진다.

우측 geth 단말에서 보듯이 대기중 transaction hash 값이 서로 일치한다.
마이닝을 하고 나면 contract 주소가 주어진다.

```
INFO [01-08|11:37:07.548] Submitted contract creation
fullhash=0x2ef00671e310f594dca636acbc352ce61463e78110e8098b0d483c8374643248
contract=0x96d8b65B4586B090840d6587C216D4A69071F405
```

![alt text](figures/sol_helloV1Web3DeployMining.png "deploy hello.sol using web3 with mining")

### REMIX에서 이벤트 확인하기

* Environment를 Javascript VM으로
* Deploy버튼 옆에 생성자 함수를 호출하기 위해 문자열 "Hello from web3"을 넣는다. 문자열에 따옴표를 잊지 않는다.
* Deploy버튼을 누르면 "Deployed Contracts" 아래, Hello1 객체가 나타난다. 괄호안 메모리는 실제 블록체인에 배포되지 않았다는 의미 (Javascript VM이므로)
* 속성 및 함수 버튼들을 눌러 결과가 출력되는지 보자.
* setHello() 함수 버튼을 누르면 화면 우측 하단을 잘 살펴보자. 특히 굵은 글씨 "logs"를 누르면 하단에 표가 펼쳐지고 이벤트 로그를 볼 수 있다.

![alt text](figures/sol_eventREMIX.png "geth download page")

### 단계 4: 사용

이벤트는 transaction log에 저장이 된다.
이벤트는 거래가 마이닝되어 블럭에 포함되어야만 발생된다.
트랙잭션이 pending되어 완료가 되지 않으면, 로그에 포함되지 않기 때문에 여기서 이벤틑가 발생하는지 지켜보는 경우 대기하게 된다.
단 이벤트가 발생하는 순서는 지켜진다.
단순하게 블록체인의 transaction이 아닌 ```call()```은 이벤트를 발생하지 않게 된다.

In [1]:
%%writefile src/Hello1UseFromFile.js
var Web3=require('web3');
//var web3=new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8345"));       //nok
var web3 = new Web3(new Web3.providers.WebsocketProvider("http://117.16.44.45:8345"));  //ok
//var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://117.16.44.45:8345"));  //ok
var fs=require('fs');
var _str = fs.readFileSync("src/Hello1.json");
var _json = JSON.parse(_str)
//var _abiArray = JSON.parse(_json.contracts.Hello1.abi);
var _abiArray = JSON.parse(_json.contracts["src/Hello1.sol:Hello1"].abi);

async function doIt() {
    var hello = new web3.eth.Contract(_abiArray, "0xA43f621f0bAFfce634E43858Fb72809f050049E4");
    var event = hello.events.PrintLog(function (error, event) {
        console.log(">>> Event fired: " + JSON.stringify(event.returnValues));
    })
    .on('>> data', function(event) {
        console.log(event);
    })
    .on('>> changed', function(event) {
        console.log(event);
    })
    .on('>> error', console.error);
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    hello.methods.sayHello().call().then(console.log);  //null
    await hello.methods.setHello().send({from: accounts[0]});
    hello.methods.sayHello().call().then(console.log);
    hello.methods.compareTo("Hello").call().then(console.log);
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    
}
doIt()


Overwriting src/Hello1UseFromFile.js


In [ ]:
!node src/Hello1UseFromFile.js

Account: 0x28E9D5b10083b74406b50cbb7B48a299BA6C8Be1
Balance before: 99942805280000000000
Hello
>>> Event fired: {"0":"0x28E9D5b10083b74406b50cbb7B48a299BA6C8Be1","1":"Hello","addr":"0x28E9D5b10083b74406b50cbb7B48a299BA6C8Be1","s":"Hello"}
Hello
true
Balance after: 99942304260000000000
Balance diff: 501019999993856


## 블록체인에서 컨트랙 삭제

블록체인에서 프로그램을 유지보수할 수 없다. 오류가 발생하거나, 쓸모가 없어지면 삭제를 해야 한다.
컨트랙은 Read Only로서, 만들어지면 영구히 남게된다.
selfdestruct()는 잔고를 전액 반환하고, 컨트랙을 블록체인으로부터 제거한다.

###  단계 1: 컨트랙 개발

In [42]:
%%writefile src/Hello2.sol
pragma solidity ^0.6.0;
contract Hello2 {
    string hello;
    address payable owner;

    event PrintLog(string _s);
    constructor() public {
        owner = msg.sender;
    }
    function sayHello() view public returns(string memory) {
        return hello;
    }
    function setHello(string memory _hello) public payable {
        hello = _hello;
        emit PrintLog(_hello);
    }
    function getBalance() view public returns(uint) {
        return address(this).balance;
    }
    function kill() public { if (msg.sender == owner) selfdestruct(owner); }
}

Overwriting src/Hello2.sol


### 단계 2: 컴파일

In [1]:
!solc src/Hello2.sol --combined-json abi,bin > src/Hello2.json

In [44]:
!cat src/Hello2.json

{"contracts":{"src/Hello2.sol:Hello2":{"abi":"[{\"inputs\":[],\"stateMutability\":\"nonpayable\",\"type\":\"constructor\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"internalType\":\"string\",\"name\":\"_s\",\"type\":\"string\"}],\"name\":\"PrintLog\",\"type\":\"event\"},{\"inputs\":[],\"name\":\"getBalance\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"kill\",\"outputs\":[],\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"sayHello\",\"outputs\":[{\"internalType\":\"string\",\"name\":\"\",\"type\":\"string\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[{\"internalType\":\"string\",\"name\":\"_hello\",\"type\":\"string\"}],\"name\":\"setHello\",\"outputs\":[],\"stateMutability\":\"payable\",\"type\":\"function\"}]","bin":"608060405234801561001057600080fd5b5033600160006101000a81548173ffffffffffffffffffffffffff

### 단계 3: 배포

컴파일한 후 abi, bin을 저장한 파일에서 읽어서 배포해보자.


In [45]:
%%writefile src/Hello2DeployFromFile.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/Hello2.json");
var _json=JSON.parse(_str)
//var _abiArray=JSON.parse(_json.contracts.sHello2.abi);
var _abiArray=JSON.parse(_json.contracts['src/Hello2.sol:Hello2'].abi);
//var _bin=_json.contracts.sHello2.bin;
var _bin="0x"+_json.contracts['src/Hello2.sol:Hello2'].bin;

//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()


Overwriting src/Hello2DeployFromFile.js


In [1]:
!node src/Hello2DeployFromFile.js

Deploying the contract from 0x28E9D5b10083b74406b50cbb7B48a299BA6C8Be1
hash: 0x8ae619bbf0f84a108b17e8796131c5160e4210e5e1060393ef71b037478614ca
---> The contract deployed to: 0x8DeCdBfD91C84F9669b826B25598BF96eEf74185


### 단계 4: 사용

In [1]:
%%writefile src/Hello2UseFromFile.js
var Web3=require('web3');
//var web3=new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8345"));       //nok
var web3 = new Web3(new Web3.providers.WebsocketProvider("http://117.16.44.45:8345"));  //ok
//var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://117.16.44.45:8345"));  //ok
var fs=require('fs');
var _str = fs.readFileSync("src/Hello2.json");
var _json=JSON.parse(_str)
//var _abiArray=JSON.parse(_json.contracts.sHello2.abi);
var _abiArray=JSON.parse(_json.contracts['src/Hello2.sol:Hello2'].abi);

async function doIt() {
    var hello = new web3.eth.Contract(_abiArray, "0x8DeCdBfD91C84F9669b826B25598BF96eEf74185");
    var event = hello.events.PrintLog(function (error, result) {
        if (!error) {
            console.log("Event fired: " + JSON.stringify(result.returnValues));
        }
    });
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    hello.methods.sayHello().call().then(console.log);  //null
    await hello.methods.setHello("Hello World!").send({from: accounts[0], value: 1111});
    hello.methods.sayHello().call().then(console.log);
    //hello.methods.getBalance().call(function(err, bal) { console.log("Contract Balance: "+bal) });
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    //hello.methods.kill().send({from: accounts[0]})
}

doIt()


Overwriting src/Hello2UseFromFile.js


kill 함수를 마지막에 호출하기 때문에 한 번 실행해서 결과를 출력한 후, 재호출하면 ```Error: Returned values aren't valid, did it run Out of Gas?```가 호출된다.

In [ ]:
!node src/Hello2UseFromFile.js

Account: 0x28E9D5b10083b74406b50cbb7B48a299BA6C8Be1
Balance before: 99933007819999997778
Hello World!
Event fired: {"0":"Hello World!","_s":"Hello World!"}
Hello World!
Balance after: 99932487159999996667
Balance diff: 520659999997952


## 무작위 수의 생성

블록체인에서 무작위 수를 생성하기 위해서는 전역변수를 활용해야 한다.
전역변수를 256비트의 해시값으로 만들고, modulus를 이용해 일정 범위의 수가 생성되도록 한다.

일련의 무작위 수를 생성하기 위해서는 전역변수 값이 짧은 시간에 변동하지 않으므로,
매 번 다른 수가 생성되도록 하려면, 약간의 트릭을 써서 카운터 값을 더해주도록 한다.

In [4]:
%%writefile src/Random.sol
pragma solidity ^0.6.0;
contract Random {
    function rand() public view returns(bytes32) {
        return keccak256(abi.encodePacked(block.timestamp, block.difficulty));
    }
    function rand0and250() public view returns(uint8) {
        return uint8(uint256(keccak256(abi.encodePacked(block.timestamp, block.difficulty)))%251);
    }
    function rand0and9() public view returns(uint8) {
        return uint8(uint256(keccak256(abi.encodePacked(block.timestamp, block.difficulty)))%10);
    }
    function rand0and2() public view returns(uint8) {
        return uint8(uint256(keccak256(abi.encodePacked(block.timestamp, block.difficulty)))%3);
    }
    function genRandomInteger() view public returns(uint8[] memory) {
        uint8[] memory r = new uint8[](10);
        for(uint i = 0; i<r.length; i++)
            //r[i] = uint8(uint256(keccak256(abi.encodePacked(block.timestamp, block.difficulty)))%10);
            //change timestamp by adding i, otherwise all the same numbers generated
            r[i] = uint8(uint256(keccak256(abi.encodePacked(block.timestamp + i, block.difficulty)))%10);
        return r;
    }
}


Overwriting src/Random.sol


## 9.7 데이터구조

### 9.7.1 array

배열은 순서가 있는 데이터로서 인덱스를 사용하여 요소를 특정할 수 있다.
컴파일 시점에 메모리가 할당되는 고정배열과 실행시점에 메모리가 배정되는 동적배열로 구분할 수 있다.

#### 고정배열

고정배열은 배열이 저장하는 데이터의 개수를 사전에 정할 수 있는 경우에 사용한다.
우측은 요소가 3개로 정해져 있는 배열이므로 좌측의 고정배열에 할당할 수 있다.

```python
string[3] cities1=["Seoul", "Sydney", "Tokyo"];
```


#### 동적배열

저장하는 데이터의 개수를 미리 정할 수 없고 나중에 정해진다.
동적배열은 ```new``` 명령어로 생성할 수 있고, 괄호 안에 개수를 적어서 초기화할 수 있다.

```python
int[] memory num=new int[](3);
```

```string```은 그 자체로 동적배열이다.
동적배열은 storage함수에만 사용할 수 있고, memory 변수에는 사용할 수 없다.
storage 배열에는 push() 함수를 사용하여 데이터를 추가할 수 있다.

```python
string[] cities2;
cities2.push("New York");
cities2.push("Beijing");
```

#### 다차원배열

정수의 동적 2차원배열이다.
앞의 3은 요소의 개수를, 뒤는 배열의 개수를 의미한다.
다음 예는 3개의 요소를 가진 배열이 2개라는 선언이다.

```python
uint[3][] marks=[[100, 80, 95],[20,30,40]];
```

### 배열의 동작

* **동적 배열은 그 크기를 알 수 없으므로 반환을 할 수 없다**. 배열을 반환하는 경우 사전에  크기를 알아야 한다.
또는 ```pragma experimental ABIEncoderV2;```라고 프로그램 앞에 선언해주어야 한다.

* 배열을 삭제하는 경우 특정 인덱스를 정해서 ```delete array[index]```.
그러나 그 데이터 항목은 그대로 유지된다는 점에 주의해야 한다.
예를 들어, 배열 ```data``` ```[1, 2, 3, 4]```에서 ```delete data[2]```하면 ```[1, 2, 0, 4]```가 된다.

* ```length```를 사용하여 배열 크기를 알 수 있다.
* ```push```는 ```storage``` 배열에만 사용하고 (즉 상태변수의 배열), ```memory``` 배열에는 사용할 수 없다.

## 실습: 배열

### 단계 1: 컨트랙 개발

함수 명 | 설명
-----|-----
```getDynamicArrMemory()``` | 메모리에 선언된 동적배열이지만 데이터를 채우고 크기를 알게 되면 반환가능 ```uint[] memory```
```getStringDynamicArrMemory()``` | ```string```은 그 자체로 동적배열이므로 반환 불가능 ```string[] memory```
```getCities1_()``` | ```string```은 그 자체로 동적배열이므로 반환 불가능 ```string[] memory```
```getCities1()``` | ```string``` 1개씩은 반환 가능
```getCities2()``` | ```pragma experimental ABIEncoderV2```를 사용하면 ```string``` 동적 배열 반환가능
```getMathAbove70()``` | 크기를 결정하지 않고, 값을 할당할 수 없슴


In [69]:
%%writefile src/ArrayTest2.sol
pragma solidity ^0.6.0;
pragma experimental ABIEncoderV2;

contract ArrayTest2 {
    string[3] cities1=["Seoul", "Sydney", "Tokyo"];
    string[] cities2 = new string[](2);
    string[] cities3;
    int[5] mathMarks;
    uint[3][] marks=[[100, 80, 95],[20,30,40]];

    function getDynamicArrMemory() pure public returns(uint[] memory) {
        uint[] memory num=new uint[](3);  //dynamic
        for (uint i=0; i<num.length; i++)
            num[i]=i;       //push() not allowed for array memeory
        return num;
    }
    //string is a dynamic array itself, which can not be returned.
    //function setArrMemory() view public returns(string[] memory) {
    function getStringDynamicArrMemory() pure public {
        //array storage not allowed
        //error: string[2] storage places = ["9000", "Sydney"];
        string[2] memory places = ["9000", "Sydney"];
        //array memory push not allowed
        //places.push("Seoul");
        places[0]="Seoul";
        //return places;
    }
    /*returning 'string[] storage' is not allowed
    function getCities1_() view public returns(string[] memory) {
        return cities1;  //can not return stoarge var. 
    }*/
    function getCities1() view public returns(string memory) {
        return cities1[0];
    }
    function getCities1Length() view public returns(uint) { return cities1.length; }
    function setCities23() public {
        cities2[0]="New York";
        cities2.push("Busan");
        cities3.push("New York");
        cities3.push("Beijing");
    }
    //dynamic array return needs "pragma experimental ABIEncoderV2;"
    function getCities2() view public returns(string[] memory){
        return cities2;
    }
    function setMathMarks() public {
        mathMarks=[100,60,95,50,80];
    }
    //run setMathMarks() beforehand
    function getMathAbove70_() view public returns(int[] memory) {
        // size is not allocated yet -> invalid opcode error
        int[] memory mathAbove70;
        uint counter = 0;
        for(uint8 i=0;i<mathMarks.length;i++)
            if(mathMarks[i]>70) {
                mathAbove70[counter] = mathMarks[i];
                //mathAbove70.push(mathMarks[i]);
                counter++;
            }
        return mathAbove70;
    }
    //run setMathMarks() beforehand
    function getMathAbove70() view public returns(int[] memory) {
        //compute lengthOfMathAbove70
        uint8 counter=0;
        uint8 lengthOfMathAbove70=0;
        for(uint8 i=0;i<mathMarks.length;i++)
            if(mathMarks[i]>70) counter++;
        lengthOfMathAbove70=counter;
        //move math marks above 70
        int[] memory mathAbove70=new int[](lengthOfMathAbove70);
        counter=0;
        for(uint i=0;i<mathMarks.length;i++) {
            if(mathMarks[i]>70) {
                mathAbove70[counter]=mathMarks[i];
                counter++;
            }
        }
        return mathAbove70;
    }
    function updateMarks() public returns(uint[3][] memory){
        marks[0][0]=90;
        return marks;
    }
    function getMarksArr() view public returns(uint[3][] memory) {
        return marks;
    }
    function getMarksLength() view public returns(uint) {
        return marks.length;
    }
}

Writing src/ArrayTest2.sol


### 단계 2: 컴파일

In [2]:
!solc src/ArrayTest2.sol --combined-json abi,bin > src/ArrayTest2.json

### 단계 3: 컨트랙 배포

#### estimateGas

배열은 ```gas```를 많이 필요로 한다. ```Gas```를 산정해보니, 앞서 컨트랙보다 많이 1,226,648가 발생하고 있다.
1,226,647로 1을 줄여서 하면 ```out of gas``` 오류가 발생한다.

```python
new web3.eth.Contract(_abiArray).deploy({data: _bin}).estimateGas(function(err, gas) {
        if(!err) console.log(">> gas: "+ gas);});
```


In [19]:
%%writefile src/ArrayTest2Deploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/ArrayTest2.json");
var _json = JSON.parse(_str)
//var _abiArray = JSON.parse(_json.contracts.sHello2.abi);
var _abiArray = JSON.parse(_json.contracts["src/ArrayTest2.sol:ArrayTest2"].abi);
//var _bin = _json.contracts.sHello2.bin;
var _bin="0x" + _json.contracts["src/ArrayTest2.sol:ArrayTest2"].bin;

//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);

    new web3.eth.Contract(_abiArray).deploy({data: _bin}).estimateGas(function(err, gas) {
        if(!err) console.log(">> gas: "+ gas);
    });

    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1226648}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}

deploy()


Overwriting src/ArrayTest2Deploy.js


In [31]:
!node src/ArrayTest2Deploy.js

Deploying the contract from 0x64893d897E16687921f7e59E68157BeA57eC6E1a
>> gas: 1226648
hash: 0xc87a8d09a695e0c57b60c97877fe920feb0668b99553114bda222600044fec58
---> The contract deployed to: 0x80A6be863994d05aB7ca6f2A9416B5eda707bBa5


### 단계 4: 사용

#### estimateGas

gas를 없이 트랙잭션을 발생하면, ```out of gas``` 오류가 발생한다.
```python
await arr.methods.setCities23().send({from: accounts[0]});  //out of gas error
```

```estimateGas()``` 함수를 사용하여 ```gas```를 산정하면 130312가 소요된다.
그 값을 넣어주면 오류가 없어지게 된다.

```python
await arr.methods.setCities23().send({from: accounts[0], gas:130312});
```

In [32]:
%%writefile src/ArrayTest2Use.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8345"));       //ok
var fs=require('fs');
var _str = fs.readFileSync("src/ArrayTest2.json");
var _json = JSON.parse(_str)
//var _abiArray = JSON.parse(_json.contracts.sHello2.abi);
var _abiArray = JSON.parse(_json.contracts["src/ArrayTest2.sol:ArrayTest2"].abi);

async function doIt() {
    var arr = new web3.eth.Contract(_abiArray, "0x80A6be863994d05aB7ca6f2A9416B5eda707bBa5");
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    
    arr.methods.setCities23().estimateGas(function(err, gas) {
        if(!err) console.log(">> gas: "+ gas);
    });

    //await arr.methods.setCities23().send({from: accounts[0]});   //out of gas error
    await arr.methods.setCities23().send({from: accounts[0], gas:130312});
    arr.methods.getCities2().call().then(console.log);

    arr.methods.setMathMarks().estimateGas(function(err, gas) {
        if(!err) console.log(">> gas: "+ gas);
    });

    await arr.methods.setMathMarks().send({from: accounts[0], gas: 122209});
    //ERROR invalid opcode arr.methods.getMathAbove70().call().then(console.log);
    arr.methods.getMarksArr().call().then(console.log);

    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    
}

doIt()


Overwriting src/ArrayTest2Use.js


In [33]:
!node src/ArrayTest2Use.js

Account: 0x64893d897E16687921f7e59E68157BeA57eC6E1a
Balance before: 99727509020000000000
>> gas: 130312
[ 'New York', '', 'Busan' ]
>> gas: 122209
[ [ '100', '80', '95' ], [ '20', '30', '40' ] ]
Balance after: 99722458600000000000
Balance diff: 5050419999997952


```gas``` 비용은 ```ArrayTest2Use.js```를 다시 실행하면 많이 낮아진다.
첫 회의 실행에서는 동적배열을 생성하고, 다음 실행에서는 그 비용이 줄어들기 때문에 ```gas```비가 낮아지고 있다.

In [34]:
!node src/ArrayTest2Use.js

Account: 0x64893d897E16687921f7e59E68157BeA57eC6E1a
Balance before: 99722458600000000000
>> gas: 95769
[ 'New York', '', 'Busan', 'Busan' ]
>> gas: 23209
[ [ '100', '80', '95' ], [ '20', '30', '40' ] ]
Balance after: 99720079040000000000
Balance diff: 2379559999995904


#### 배열의 출력

```web3.js``` 현재 버전에서는 Solidity -> node로 배열의 전달에 문제가 없어서 출력이 올바르게 되고 있다.
이전 버전에서는 배열을 올바르게 출력하려면 ```toString()``` 함수를 사용해야 한다.

```python
> arraytest.getMarksArr()                      출력하기 위해서는 toString()
[ [ BigNumber { s: 1, e: 2, c: [Array] },
    BigNumber { s: 1, e: 1, c: [Array] },
    BigNumber { s: 1, e: 1, c: [Array] } ],
  [ BigNumber { s: 1, e: 1, c: [Array] },
    BigNumber { s: 1, e: 1, c: [Array] },
    BigNumber { s: 1, e: 1, c: [Array] } ] ]
> arraytest.getMarksLength()
BigNumber { s: 1, e: 0, c: [ 2 ] }
> arraytest.getMarksLength().toString()
'2'
> arraytest.getMarksArr().toString()
'100,80,95,20,30,40'
```

## 실습: Members 배열

배열을 검색하는 것은 비용이 많이 발생한다. 
mapping을 사용하거나, 클라이언트 측에서 배열 데이터를 검색하는 편으로 하자.

문자열 비교 '=='는 사용할 수 없다. ```keccak256()``` 해싱을 한 후 비교해야 한다.

In [2]:
%%writefile src/Members.sol
pragma solidity ^0.5.0;
contract Members{
    address owner;
    event printAddr(address arg);
    struct Member{
        uint id;
        string name;
    }
    Member[] public memberArr;
    constructor() public {
        owner=msg.sender;
    }
    function del() public {
        delete memberArr;
    }
    function delOne(uint i) public{
        delete memberArr[i];  //try pop()
    }
    function add(uint id,string memory name) public {
        memberArr.push(Member(id,name));
    }
    //return Member
    function getMember(string memory who) view public returns(uint, string memory) {
        uint _id;
        string memory _name;
        for(uint i=0;i<memberArr.length;i++) {
            _name=memberArr[i].name;
            if(keccak256(abi.encodePacked(_name))==keccak256(abi.encodePacked(who))) {
                _id=memberArr[i].id;
                _name=memberArr[i].name;
            }
        }
        return (_id,_name);
    }
    function compareStr(string memory s1, string memory s2) pure public returns(bool) {
        return keccak256(abi.encodePacked(s1))==keccak256(abi.encodePacked(s2));
    }
    function compareBytes(bytes memory b1, bytes memory b2) pure public returns(bool) {
        return keccak256(b1) == keccak256(b2);
    }
    function getLength() view public returns(uint) {
        return memberArr.length;
    }
}

Writing src/Members.sol


### 9.7.2 mapping
<a id='S.5.2'></a>

```mapping```은 `키`와 키에 해당하는 `값`을 쌍으로 저장한다. **```=>```** 기호를 사용하여, 좌측에 키를 오른쪽에 값을 적어준다.
다음 ```mapping``` 코드를 보면, ```string```은 키로 값은 ```uint```로 정의 하고, 그 변수 명은 ```balances```로 선언하고 있다.
그리고 ```string```으로 선언된 키에 ```jsl```, ```uint```로 선언된 값에 잔고 100을 저장한다.

```python
mapping (string => uint) public balances;
balances["jsl"] = 100;
```

## 실습: 은행

앞서 은행에서는 한 계정에서의 입금, 출금을 실행해 보았다.
여러 계정이 있는 경우, mapping을 이용해 잔고를 관리할 수 있다.

```mapping(address=>uint) balanceOf;``` 라고 선언하고
```balanceOf[<address>]=amount;```로 잔고를 저장할 수 있다.

증액, 감액을 하려면:
```python
uint balanceToAdd=111;
balanceOf[<address>] += balanceToAdd;
```

* adddress 자신의 주소를 따옴표 없이, 금액도 따옴표 없이 정수

입출금은 기능을 어떻게 설계하느냐에 따라 금액변동에 영향을 미칠 수 있기 마련이다.
여기서는 입출금 및 계좌이체를 하면 자신의 계좌, owner 계좌 그리고 컨트랙의 계좌에 아래와 같이 변동이 있도록 해보았다.

함수 | 관련 잔고 | owner잔고 | 컨트랙 잔고
-----|-----|-----|-----|-----|
deposit(111)을 하면 | owner잔고에 입금함 | owner 잔고 111 증가 | 컨트랙잔고 111 증가
forwardTo(0x1234..., 11)을 하면| 계좌이체이므로 Ox1234... 잔고 11 증가 | owner잔고 11감소 | 컨트랙 잔고 변동 없슴
withdraw(0x1234..., 11)을 하면 | Ox1234... 잔고 11 감소 | owner잔고 변동 없슴 | 컨트랙 잔고 11감소

In [1]:
%%writefile src/BankV4.sol
//check for Coin.sol (ch12) and BankV3.sol 201906
pragma solidity ^0.6.0;
contract BankV4 {
    address owner;
    mapping (address => uint) balanceOf;
    constructor() public {
        owner = msg.sender;
    }
    // save to individual addresses
    function deposit(uint amount) payable public onlyOwner {
        require(msg.value == amount);
        balanceOf[msg.sender] += amount;
    }
    // forward from owner to another
    function forwardTo(address receiver, uint amount) payable public onlyOwner {
        require(balanceOf[msg.sender] >= amount);
        balanceOf[msg.sender] -= amount; // Subtract from the sender
        balanceOf[receiver] += amount;   // Add the same to the recipient
    }
    function withdraw(address payable receiver, uint amount) public onlyOwner {
        require(balanceOf[receiver] >= amount && address(this).balance >= amount);
        balanceOf[receiver] -= amount;
        receiver.transfer(amount);
    }
    function getBalance() public view returns(uint, uint) {
        return (address(this).balance, balanceOf[owner]);
    }
    function getBalanceOf(address addr) public view returns (uint) {
        return balanceOf[addr];
    }
    modifier onlyOwner {
        require(msg.sender == owner);
        _;
    }
}

Overwriting src/BankV4.sol


* 중복키 확인

맵은 그 특성상 중복키는 허용되지 않는다. 동일한 키로 당연히 데이터를 입력할 수가 없다.
맵에 이미 키가 존재하는지 확인할 수 있는 ```containsKey()```와 같은 지원 함수가 없다.
필드 ```isMember```를 설정해놓고 그 값을 확인하는 방법으로 하자.
```
mapping(address=>Member) memberMap;
struct Member {
	uint id;
	string name;
	bool isMember;
}
if (memberMap(<address>).isMember) 중복키 존재함
```

* 삭제

키삭제는 쉽지 않다. 다른 언어에서는 ```remove()```함수가 지원되어, 간편하게 키를 지울 수 있는 것과 비교된다.

```
delete pendingAccessRequest[key];
```

#### bidirectional map

주소에 대해 회원(아이디, 이름)을 저장하는 ```map```이다.
이 경우 'id' 또는 '이름'으로 '주소'를 검색하는 기능은 ```bidiretional map```으로 구현하였다.
이렇게 하면, 반복문을 사용하지 않고 검색하는 기능이 가능해 진다.
특정 id에 해당하는 주소를 찾고, 그 주소에 해당하는 값을 읽어올 수 있게 된다.

줄 | 설명
-----|-----
9 | 주소의  ```Member strcut ```을 조회하는 map
10 | 이름의 주소를 조회하는 map
11~19 ```addMember() ``` |  ```Member struct```을 추가하고 ```getMemberByName()``` 함수에서 이름을 입력하면 주소를 조회할 수 있도록 저장. ```name```이 중복되는 경우, 엎어쓰기 때문에 문제가 될 수 있다. 실제는 중복이 없는 ```id```를 사용하여 ```bidirectional map```을 구현해야 한다.
20~24 ```getMemberById()``` | bidirectional map에서 ```id```에 해당하는 주소를 조회하고, 그 주소키를 이용 Member를 조회
25~28 ```getMemberByName()``` | bidirectional map에서 ```name```에 해당하는 주소를 조회
29~32 ```getMember()``` | 주소에 해당하는 ```Member struct```을 조회

In [3]:
%%writefile src/MembersMap.sol
pragma solidity ^0.6.0;

contract MembersMap {
    struct Member {
        uint id;
        string name;
    }
    //bidrectional map
    mapping(address=>Member) memberMap;
    mapping(uint=>address) addressById;
    mapping(string=>address) addressByName;
    function addMember (uint _id, string memory _name) public {
       Member memory m=Member(_id, _name);
       memberMap[msg.sender]=m;
       //saving into a bidiretional map to get address by id
       addressById[_id]=msg.sender;
       //saving into a bidiretional map to get address by name
       addressByName[_name]=msg.sender;
    }
    //get Member by id
    function getMemberById(uint _id) view public returns(uint, string memory) {
        Member memory my = memberMap[addressById[_id]];
        return (my.id, my.name);
    }
    // get address (not Member) by name
    function getMemberAddressByName(string memory _name) public view returns(address) {
        return addressByName[_name];
    }
    function getMember(address addr) view public returns (uint, string memory) {
        Member memory m=memberMap[addr];
        return (m.id, m.name);
    }
}

Overwriting src/MembersMap.sol


## 9.8 Exception
<a id='S.6'></a>

오류는 발생하기 마련이다. 컴파일 오류는 수정하기 비교적 용이하다. 그러나 실행시점에 발생하는 오류는 잡아내기 어렵다.
다른 언어에서 보통 제공하는 ```try..catch``` 구문도 제공되지 않는다.
버전 4.10 이전에는 ```throw``` 명령문이 사용되었지만 문제가 있었다.
지급한 gas를 다 소비하고 오류발생 이전의 상태로 돌려놓았다.

```python
if (msg.sender != owner) { throw; }
```

버전 4.10이후에는 ```throw```구문은 더 이상 사용되지 않고 다음 구문이 사용된다.


발생하는 예외는 여러 경우가 있을 수 있다. 실행 시점에 발생하는 예외는 배열이 범위를 넘어서거나 ```out-of-gas errors```, ```divide by zero```와 같이 어떤 특정 조건에서 예외를 발생시킬 수 있다.
이 경우 실행이 중지되고, 원상복구 (상태변수와 잔고)된다.

###  require
```require()```문은 자주 쓰이는 명령문으로 실행한 결과를 ```true```, ```false```로 반환된다. ```false```인 경우 예외가 발생하고 실행이 중지된다. 미사용 gas를 호출자에게 반환하고 상태를 원래대로 복원한다.
* 송금이 되었는지 확인하는 경우 ```require(<address>.transfer(amount))```
* 조건이 충족되었는지 확인하는 경우 ```require(msg.value == amount)```, ```require(address(this).balance >= amount)```
* 함수의 진행에 필요한 사전조건을 확인하는 방식으로, 보통 함수의 맨 앞에 위치하여 사용된다.

### revert
```revert()```문은 ```require()```함수와 동일한 의미지만, ```revert()```문 내에서 참인지, 거짓인지 어떤 평가를 하지 않는다는 점이 다르다. 프로그램 로직에 대해 이진비교 이상을 하는 경우에 사용되기 때문에 **```if/else```**문과 많이 사용된다. ```require()```와 마찬가지로 ```revert()``` 명령문은 미사용 gas를 호출자에게 반환하고 상태를 원래대로 복원한다.

### assert
```assert()``` 미사용 gas는 반환되지 않고, 모두 소모된다. 상태는 원래대로 복원된다.
* overflow/underflow ```assert(age > 20)```
* 항상 충족해야 하는 조건 ```assert(address(this).balance >= 0)```
* 함수가 끝나고 나서 사후조건을 확인하는 방식으로, 보통 함수의 맨 뒤에 위치하여 사용된다. ```require```에 비해 자주 사용되지 않는다.

명령문 | gas 반환 | 원복 | 사용하는 경우
-----|-----|-----|-----
```assert()``` | NO | YES | overflow/underflow, invariants 등 어떤 오류가 발생하는 것을 예방하는 경우. 실패하면 중단되며, 다음으로 진행되지 않는다.
```revert()``` | YES | YES | ```require()```는 참, 거짓을 평가하지만 조건문 없이 수행되기 때문에 이진비교 이상을 하는 경우. ```revert```되면 중단되며, 다음으로 진행되지 않는다.
```require()``` | YES | YES | 사용자 입력 , 상태변수 값, 반환 값 등 조건을 검증하는 경우. 실패하면 중단되며, 다음으로 진행되지 않는다.

In [3]:
%%writefile src/ExceptionTest.sol
pragma solidity *0.6.0;

contract ExceptionTest {
    address owner;
    constructor() public {
        owner=msg.sender;
    }
    function requireException() view public returns(string memory) {
        //if (msg.sender != owner) { throw; }
        require(msg.sender != owner, "Sorry! You are owner. Require failed...");
        return "require condition met";
    }
    function assertException() view public returns(string memory) {
        assert(msg.sender == owner);
        return "asserted";  //do this line only 'assert' is succeeded
    }
    function revertException() view public returns(string memory) {
        if(msg.sender != owner) {
            revert("Sorry! You are NOT owner. Reverted...");
            //return "reverted";  // this line can't be reached if revert is executed 
        } else {
            return "not reverted";
        }
    }
    function raiseException() pure public {
        int x=0;
        int y=0;
        x/y;    //divide by zero
    }
}

Overwriting src/ExceptionTest.sol


## 연습문제 주문

사용자별 주문은 1:1 관계로 구현하고 있다 (1:n 관계로 구현하려면 복잡해진다). 주문id로 조회할 수 있도록 하기 위해 bidiectional map을 사용하였다.

줄 | 설명
-----|-----
4~12 | 주문 데이터, ```isOrdered```는 중복키 확인용도로 쓰인다.
14 | 주문 갯수. 반복문으로 갯수를 세는 것을 피하기 위해서 계속 추적한다.
15 | 주문 총액 역시 반복문을 피하기 위해 추적한다.
18 | 주문번호에서 검색을 하기 위해 설정한다.
22~30 ```placeOrder()``` | 주문금액 확인, 전송자 주소로 주문데이터 확인, 주문갯수 및 주문총액을 추적. bidirectional map을 이용해서 주문id 별로 주문내역을 검색할 수 있게 함. 주문id는 중복되지 않게 생성을 하도록 함.
32~35 ```updateStatus()``` | 주문번호를 이용해 주문의 상태를 갱신한다. 관리자만 갱신권한을 가진다.


## 연습문제

A, B 2인의 가위바위보 게임 후 게임머니를 이긴 사람에게 지급